In [1]:
import torch
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))
    print("CUDA version:", torch.version.cuda)
else:
    print("GPU not detected")


CUDA available: True
GPU Name: NVIDIA GeForce MX130
CUDA version: 11.8


In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import open3d as o3d
import threading
from queue import Queue
import time

fisheye_params = {
    "width": 2592,
    "height": 1944,
    "cx": 1296.0,
    "cy": 972.0,
    "radius": 952.56,
    "theta_max_rad": 1.5708
}

cam_settings = [
    {"cx": 1296.0, "cy": 972.0, "fov": 90.0, "pitch": 58.0, "yaw": -18.0},
    {"cx": 1296.0, "cy": 972.0, "fov": 90.0, "pitch": 54.0, "yaw": 48.0},
    {"cx": 1296.0, "cy": 972.0, "fov": 90.0, "pitch": 28.0, "yaw": 150.0},
    {"cx": 1296.0, "cy": 972.0, "fov": 90.0, "pitch": 58.0, "yaw": 294.0},
]

views_path = [
    "captures_yaw_pitch/capture1/view1_1.jpg",
    "captures_yaw_pitch/capture1/view2_1.jpg",
    "captures_yaw_pitch/capture1/view3_1.jpg",
    "captures_yaw_pitch/capture1/view4_1.jpg"
]
fisheye_path = "captures_yaw_pitch/capture1/fisheye_1.jpg"

#undistort to BEV
BEV = cv2.imread("ohlf.png")   
homography_BEV = np.load("Homography for 229_camera_.npy")
dis_rect_w, dis_rect_h = 2592,2592
fov = 160.0
fov_RAD = np.deg2rad(fov)
focal = (dis_rect_w / 2) / np.tan(fov_RAD / 2)

dis_K_rect = np.array([[focal, 0.0, dis_rect_w / 2],
                   [0.0, focal, dis_rect_h / 2],
                   [0.0, 0.0, 1.0]])

f_fish = fisheye_params["radius"] / fisheye_params["theta_max_rad"]
K_fish = np.array([[f_fish, 0.0, fisheye_params["cx"]],
                   [0.0, f_fish, fisheye_params["cy"]],
                   [0.0, 0.0, 1.0]])
D_fish = np.zeros((4, 1)) 

#3d rendering
added_spheres = []
point_queue = Queue()
homography_BEV_to_3D = np.load("Homography for bev to 3d.npy")
mesh = o3d.io.read_triangle_mesh("OHLF_obj/OHLF_v2.8.3 (1).obj", enable_post_processing=True)
mesh.compute_vertex_normals()
vis = o3d.visualization.Visualizer()
vis.create_window(window_name="3D Model", width=1280, height=720)
vis.add_geometry(mesh)
opt = vis.get_render_option()


def load_images(views, fisheye_path=None):
    image_bgr = []
    for p in views:
        b = cv2.imread(p)
        if b is None:
            raise FileNotFoundError(f"Missing {p}")
        image_bgr.append(b)
    fisheye_img = None
    if fisheye_path:
        fisheye_img = cv2.imread(fisheye_path)
        
    return image_bgr, fisheye_img
views, fisheye_img = load_images(views_path, fisheye_path)
rect_h, rect_w = 720,1280

def compute_basis(yaw_deg, pitch_deg):
    yaw_deg_corrected = yaw_deg - 90.0
    yaw, pitch = np.deg2rad([yaw_deg_corrected, pitch_deg])

    Rz = np.array([
        [ np.cos(-yaw), -np.sin(-yaw), 0],
        [ np.sin(-yaw),  np.cos(-yaw), 0],
        [ 0,             0,            1]
    ])
    Rx = np.array([
        [ 1, 0,           0          ],
        [ 0, np.cos(pitch), -np.sin(pitch)],
        [ 0, np.sin(pitch),  np.cos(pitch)]
    ])
    R_ = Rx @ Rz
    R = np.diag([1, -1, 1]) @ R_
    return R

def rect_to_fisheye_point(u, v, view_params, fisheye_params):
    Hr, Wr = rect_h, rect_w
    cx_r, cy_r = Wr / 2, Hr / 2
    fov = view_params["fov"]
    f = (Wr / 2) / np.tan(np.deg2rad(fov / 2))
    x = (u - cx_r) / f
    y = (v - cy_r) / f
    z = 1
    ray_rect = np.array([x, y, z])
    ray_rect /= np.linalg.norm(ray_rect)
    Rot = compute_basis(view_params["yaw"], view_params["pitch"])
    ray_fish = Rot.T @ ray_rect
    Xf, Yf, Zf = ray_fish
    theta = np.arccos(np.clip(Zf, -1, 1))
    phi = np.arctan2(Yf, Xf)
    radius, theta_max = fisheye_params["radius"], fisheye_params["theta_max_rad"]
    cx_f, cy_f = fisheye_params["cx"], fisheye_params["cy"]
    r = (theta / theta_max) * radius
    u_f = cx_f + r * np.cos(phi)
    v_f = cy_f - r * np.sin(phi)
    return (int(u_f), int(v_f))

def fisheye_to_rect_point(u_f, v_f, view_params, fisheye_params):
    cx_f, cy_f = fisheye_params["cx"], fisheye_params["cy"]
    radius, theta_max = fisheye_params["radius"], fisheye_params["theta_max_rad"]

    dx, dy = u_f - cx_f, cy_f - v_f
    r = np.sqrt(dx ** 2 + dy ** 2)
    theta = (r / radius) * theta_max
    phi = np.arctan2(dy, dx)
    sin_t = np.sin(theta)
    ray_fish = np.array([sin_t * np.cos(phi), 
                         sin_t * np.sin(phi), 
                         np.cos(theta)])
    Rot = compute_basis(view_params["yaw"], view_params["pitch"])
    ray_rect = Rot @ ray_fish

    Xc, Yc, Zc = ray_rect
    if Zc <= 0:
        return None

    Hr, Wr = rect_h, rect_w
    fov = view_params["fov"]
    f = (Wr / 2) / np.tan(np.deg2rad(fov / 2))
    cx_r, cy_r = Wr / 2, Hr / 2
    u = f * (Xc / Zc) + cx_r
    v = f * (Yc / Zc) + cy_r
    if 0 <= u < Wr and 0 <= v < Hr:
        return (int(u), int(v))
    return None

def make_rect_grid_disp():
    grid = np.ones((RECT_TILE_H_DISP * 2, RECT_TILE_W_DISP * 2, 3), dtype=np.uint8)
    grid[0:RECT_TILE_H_DISP, 0:RECT_TILE_W_DISP] = views_disp[0]
    grid[0:RECT_TILE_H_DISP, RECT_TILE_W_DISP:] = views_disp[1]
    grid[RECT_TILE_H_DISP:, 0:RECT_TILE_W_DISP] = views_disp[2]
    grid[RECT_TILE_H_DISP:, RECT_TILE_W_DISP:] = views_disp[3]
    return grid

def map_2d_to_3d(u, v, homography_bev_to_3d, y_value=0.0):
    src = np.array([u, v, 1.0], dtype=np.float64)
    dst = homography_bev_to_3d @ src
    dst /= dst[2]
    return np.array([dst[0], y_value, dst[1]])

def add_point_realtime(point3d):
    if added_spheres:
        vis.remove_geometry(added_spheres[-1], reset_bounding_box=False)
        added_spheres.clear()

    sphere = o3d.geometry.TriangleMesh.create_sphere(radius=0.25)
    sphere.paint_uniform_color([1, 0, 0])
    sphere.translate(point3d)
    vis.add_geometry(sphere, reset_bounding_box=False)
    vis.update_geometry(sphere)
    vis.poll_events()
    vis.update_renderer()
    added_spheres.append(sphere)
    



#Display
scale_BEV=0.5
scale_rect = 0.5
FISH_W_DISP, FISH_H_DISP = 1280, 720
RECT_TILE_W_DISP, RECT_TILE_H_DISP = int(rect_w * scale_rect), int(rect_h * scale_rect)
bev_display = cv2.resize(BEV, (0,0), fx=scale_BEV, fy=scale_BEV)
views_disp = [cv2.resize(v, (0,0), fx=scale_rect, fy=scale_rect) for v in views]
fisheye_display = cv2.resize(fisheye_img, (FISH_W_DISP, FISH_H_DISP))

scale_fx = fisheye_params["width"] / FISH_W_DISP
scale_fy = fisheye_params["height"] / FISH_H_DISP
rect_grid_disp = make_rect_grid_disp()

def draw_text(img, text, pos, color=(255, 255, 255)):
    font = cv2.FONT_HERSHEY_COMPLEX  
    scale = 0.8                 
    thickness = 2         
    x, y = int(pos[0]), int(pos[1])
    cv2.putText(img, text, (x + 1, y + 1), font, scale, (0, 0, 0), thickness , cv2.LINE_AA)
    cv2.putText(img, text, (x, y), font, scale, color, thickness, cv2.LINE_AA)

def run():
    def on_mouse(event, x, y, flags, param):
        global canvas
        if event != cv2.EVENT_LBUTTONDOWN:
            return

        fisheye_disp = fisheye_display.copy()
        rect_disp = rect_grid_disp.copy()
        bev_disp = bev_display.copy()

        if x < FISH_W_DISP:  
            u_full, v_full = x * scale_fx, y * scale_fy
            cv2.circle(fisheye_disp, (x, y), 3, (0, 255, 0), -1)
            draw_text(fisheye_disp, f"({u_full:.1f},{v_full:.1f})", (x, y), (0, 255, 0))
            print(f"Click Fisheye Point = ({u_full:.1f},{v_full:.1f})")

            for i, cam in enumerate(cam_settings):
                pt = fisheye_to_rect_point(u_full, v_full, cam, fisheye_params)
                if pt is not None:
                    u_rect, v_rect = pt
                    ox = (i % 2) * RECT_TILE_W_DISP
                    oy = (i // 2) * RECT_TILE_H_DISP
                    u_disp, v_disp = int(u_rect * scale_rect), int(v_rect * scale_rect)
                    pos = (ox + u_disp, oy + v_disp)
                    cv2.circle(rect_disp, pos, 3, (0, 0, 255), -1)
                    draw_text(rect_disp, f"({u_rect},{v_rect})", pos, (0, 0, 255))
                    print(f" Rect view {i+1}: Rect Point = ({u_rect:.1f},{v_rect:.1f}) ")

        else: 
            xr = x - FISH_W_DISP
            xi = xr % RECT_TILE_W_DISP
            yi = y % RECT_TILE_H_DISP
            view_idx = (y >= RECT_TILE_H_DISP) * 2 + (xr >= RECT_TILE_W_DISP)
            u_full, v_full = xi / scale_rect, yi / scale_rect
            print(f"Click Rect_point = ({u_full},{v_full}) ")
            draw_text(rect_disp, f"({u_full},{v_full})", (x - FISH_W_DISP, y), (0, 255, 0))
            cv2.circle(rect_disp, (x - FISH_W_DISP, y), 3, (0, 255, 0), -1)
            pt_f = rect_to_fisheye_point(u_full, v_full, cam_settings[view_idx], fisheye_params)
            if pt_f is not None:
                u_f, v_f = pt_f
                u_fd, v_fd = int(u_f / scale_fx), int(v_f / scale_fy)
                cv2.circle(fisheye_disp, (u_fd, v_fd), 3, (0, 0,255), -1)
                draw_text(fisheye_disp, f"({int(u_f)},{int(v_f)})", (u_fd, v_fd), (0, 0, 255))
                pt_f = np.array(pt_f)
                pt_f = pt_f.astype(np.float32).reshape(-1,1,2)
                rect_point = cv2.fisheye.undistortPoints(pt_f, K_fish, D_fish, P=dis_K_rect)
                mapped_pt = cv2.perspectiveTransform(rect_point, homography_BEV)
                mx, my = int(mapped_pt[0][0][0]), int(mapped_pt[0][0][1])
                x_disp, y_disp = int(mx * scale_BEV), int(my * scale_BEV)
                cv2.circle(bev_disp, (x_disp, y_disp), 3, (0, 0, 255), -1)
                cv2.putText(bev_disp, f"({mx}, {my})", (x_disp + 10, y_disp - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
                p3d = map_2d_to_3d(mx,my,homography_BEV_to_3D)
                p3d = [abs(v) for v in p3d]
                point_queue.put(p3d)
                print(f" Fisheye Point = ({int(u_f)},{int(v_f)}) | BEV Point = ({mx},{my}) | 3D Model = ({p3d[0]:.2f},{p3d[1]:.2f},{p3d[2]:.2f})")
                
        
        canvas = np.ones((max(FISH_H_DISP, rect_disp.shape[0]), FISH_W_DISP + rect_disp.shape[1], 3), dtype=np.uint8)
        canvas[0:FISH_H_DISP, 0:FISH_W_DISP] = fisheye_disp
        canvas[0:rect_disp.shape[0], FISH_W_DISP:] = rect_disp
        # cv2.imshow("BEV Image", bev_disp)
        cv2.imshow("Unified Mapping", canvas)
        

    canvas = np.ones((rect_h, 2560, 3), dtype=np.uint8)
    canvas[0:rect_h, 0:rect_w] = fisheye_display
    canvas[0:rect_h, rect_w:] = rect_grid_disp

    cv2.namedWindow("Unified Mapping", cv2.WINDOW_NORMAL)
    cv2.setMouseCallback("Unified Mapping", on_mouse)
    cv2.imshow("Unified Mapping", canvas)
    # cv2.imshow("BEV Image", bev_display)

    while True:
        if cv2.waitKey(20) & 0xFF == 27:
            break

    cv2.destroyAllWindows()

cv_thread = threading.Thread(target=run, daemon=True)
cv_thread.start()
try:
    while cv_thread.is_alive():
        while not point_queue.empty():
            p = point_queue.get()
            add_point_realtime(p)
        vis.poll_events()
        vis.update_renderer()
        time.sleep(0.03)
        
finally:
    vis.destroy_window()
    cv2.destroyAllWindows()

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Click Rect_point = (430.0,566.0) 
 Fisheye Point = (1748,909) | BEV Point = (1273,667) | 3D Model = (14.91,0.00,3.11)
Click Rect_point = (1014.0,334.0) 
 Fisheye Point = (1178,1646) | BEV Point = (1388,675) | 3D Model = (6.24,0.00,2.66)
Click Fisheye Point = (1743.5,953.1)
 Rect view 1: Rect Point = (477.0,562.0) 
 Rect view 2: Rect Point = (1017.0,611.0) 
Click Fisheye Point = (1500.5,909.9)
Click Fisheye Point = (1362.8,1007.1)
Click Fisheye Point = (1439.8,1188.0)
Click Fisheye Point = (1871.1,1123.2)
 Rect view 1: Rect Point = (609.0,380.0) 
 Rect view 4: Rect Point = (76.0,530.0) 
Click Fisheye Point = (1500.5,980.1)
Click Fisheye Point = (1541.0,920.7)
Click Fisheye Point = (1844.8,1063.8)
 Rect view 1: Rect Point = (563.0,424.0) 
 Rect view 2: Rect Point = (1250.0,579.0) 
 Rect view 4: Rect Point = (29.0,621.0) 


In [8]:
import numpy as np
import cv2
import open3d as o3d
import threading
from ultralytics import YOLO
from queue import Queue
import time
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")


fisheye_params = {
    "width": 2592,
    "height": 1944,
    "cx": 1296.0,
    "cy": 972.0,
    "radius": 952.56,
    "theta_max_rad": 1.5708
}

cam_settings = [
    {"cx": 1296.0, "cy": 972.0, "fov": 90.0, "pitch": 58.0, "yaw": -18.0},
    {"cx": 1296.0, "cy": 972.0, "fov": 90.0, "pitch": 54.0, "yaw": 48.0},
    {"cx": 1296.0, "cy": 972.0, "fov": 90.0, "pitch": 28.0, "yaw": 150.0},
    {"cx": 1296.0, "cy": 972.0, "fov": 90.0, "pitch": 58.0, "yaw": 294.0},
]

views_path = [
    "recordings_2/camera_1.mp4",
    "recordings_2/camera_2.mp4",
    "recordings_2/camera_3.mp4",
    "recordings_2/camera_4.mp4"
]
fisheye_path = "recordings_2/fisheye.mp4"

model = YOLO("16_09_25_SENd_v2.5_10s_960.pt").to(device)
BEV = cv2.imread("ohlf.png")
homography_BEV = np.load("Homography for 229_camera_.npy")
homography_BEV_to_3D = np.load("Homography for bev to 3d.npy")

added_spheres = []
point_queue = Queue()
mesh = o3d.io.read_triangle_mesh("OHLF_obj/OHLF_v2.8.3 (1).obj", enable_post_processing=True)
mesh.compute_vertex_normals()
vis = o3d.visualization.Visualizer()
vis.create_window(window_name="3D Model", width=1280, height=720)
vis.add_geometry(mesh)


CLASS_COLORS = {
    0: (0, 255, 0),   
    1: (255, 0, 0),   
    2: (0, 165, 255),  
}
FONT = cv2.FONT_HERSHEY_SIMPLEX

def compute_basis(yaw_deg, pitch_deg):
    yaw_deg_corrected = yaw_deg - 90.0
    yaw, pitch = np.deg2rad([yaw_deg_corrected, pitch_deg])
    Rz = np.array([[np.cos(-yaw), -np.sin(-yaw), 0],
                   [np.sin(-yaw), np.cos(-yaw), 0],
                   [0, 0, 1]])
    Rx = np.array([[1, 0, 0],
                   [0, np.cos(pitch), -np.sin(pitch)],
                   [0, np.sin(pitch), np.cos(pitch)]])
    return np.diag([1, -1, 1]) @ (Rx @ Rz)

def rect_to_fisheye_point(u, v, view_params, fisheye_params, rect_w, rect_h):
    Hr, Wr = rect_h, rect_w
    cx_r, cy_r = Wr / 2, Hr / 2
    fov = view_params["fov"]
    f = (Wr / 2) / np.tan(np.deg2rad(fov / 2))
    x = (u - cx_r) / f
    y = (v - cy_r) / f
    z = 1
    ray_rect = np.array([x, y, z])
    ray_rect /= np.linalg.norm(ray_rect)
    Rot = compute_basis(view_params["yaw"], view_params["pitch"])
    ray_fish = Rot.T @ ray_rect
    Xf, Yf, Zf = ray_fish
    theta = np.arccos(np.clip(Zf, -1, 1))
    phi = np.arctan2(Yf, Xf)
    radius, theta_max = fisheye_params["radius"], fisheye_params["theta_max_rad"]
    cx_f, cy_f = fisheye_params["cx"], fisheye_params["cy"]
    r = (theta / theta_max) * radius
    u_f = cx_f + r * np.cos(phi)
    v_f = cy_f - r * np.sin(phi)
    return (float(u_f), float(v_f))

def map_2d_to_3d(u, v, homography_bev_to_3d, y_value=0.0):
    src = np.array([u, v, 1.0], dtype=np.float64)
    dst = homography_bev_to_3d @ src
    dst /= dst[2]
    return np.array([dst[0], y_value, dst[1]])

def add_point_realtime(point3d):
    if added_spheres:
        vis.remove_geometry(added_spheres[-1], reset_bounding_box=False)
        added_spheres.clear()
    sphere = o3d.geometry.TriangleMesh.create_sphere(radius=0.25)
    sphere.paint_uniform_color([1, 0, 0])
    sphere.translate(point3d)
    vis.add_geometry(sphere, reset_bounding_box=False)
    vis.update_geometry(sphere)
    vis.poll_events()
    vis.update_renderer()
    added_spheres.append(sphere)

dis_rect_w, dis_rect_h = 2592, 2592
fov = 160.0
fov_RAD = np.deg2rad(fov)
focal = (dis_rect_w / 2) / np.tan(fov_RAD / 2)
dis_K_rect = np.array([[focal, 0.0, dis_rect_w / 2],
                       [0.0, focal, dis_rect_h / 2],
                       [0.0, 0.0, 1.0]])

f_fish = fisheye_params["radius"] / fisheye_params["theta_max_rad"]
K_fish = np.array([[f_fish, 0.0, fisheye_params["cx"]],
                   [0.0, f_fish, fisheye_params["cy"]],
                   [0.0, 0.0, 1.0]])
D_fish = np.zeros((4, 1))

TARGET_HEIGHT = 720
TARGET_WIDTH_FISHEYE = 720
TARGET_WIDTH_RECT = 720
cap_fish = cv2.VideoCapture(fisheye_path)
caps_rect = [cv2.VideoCapture(v) for v in views_path]

def read_frame_loop(cap):
    ret, frame = cap.read()
    if not ret:
        cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
        ret, frame = cap.read()
    return ret, frame

def make_rect_grid_disp_from_native(views_native):
    h, w = views_native[0].shape[:2]
    grid = np.ones((h * 2, w * 2, 3), dtype=np.uint8) * 0 
    grid[0:h, 0:w] = views_native[0]
    grid[0:h, w:2*w] = views_native[1]
    grid[h:2*h, 0:w] = views_native[2]
    grid[h:2*h, w:2*w] = views_native[3]
    return grid

def run():
    global vis
    frame_index = 0 

    while True:
        ret_f, frame_fish = read_frame_loop(cap_fish)
        if not ret_f:
            break

        views = []
        for c in caps_rect:
            ret, f = read_frame_loop(c)
            if not ret:
                break

            views.append(f.copy())
        if len(views) != 4:
            break

        frame_index += 1
        fisheye_h, fisheye_w = frame_fish.shape[:2]
        rect_h, rect_w = views[0].shape[:2]
        all_points_this_cycle = []
        all_fisheye_points = []     
        all_rect_points_native = [] 

        for i, frame in enumerate(views):
            try:
                results = model(frame, device=device, verbose=False)
            except Exception as e:
                print("Model error:", e)
                continue

            det_count = 0
            for result in results:
                if not hasattr(result, "boxes") or len(result.boxes) == 0:
                    continue

                for box in result.boxes:
                    try:
                        cls = int(box.cls.cpu().numpy()) if hasattr(box.cls, "cpu") else int(box.cls)
                    except Exception:
                        cls = int(getattr(box, "class", -1))

                    if cls not in [0, 1, 2]:
                        continue

                    try:
                        conf = float(box.conf.cpu().numpy()) if hasattr(box.conf, "cpu") else float(box.conf)
                    except Exception:
                        conf = 0.0

                    try:
                        x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                    except Exception:
                        coords = np.array(box.xyxy).reshape(-1)
                        x1, y1, x2, y2 = coords[0], coords[1], coords[2], coords[3]

                    color = CLASS_COLORS.get(cls, (0, 255, 255))
                    cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)


                    det_count += 1
                    u_full = int((x1 + x2) / 2.0)
                    v_full = int(y2)

                    pt_f = rect_to_fisheye_point(u_full, v_full, cam_settings[i], fisheye_params, rect_w, rect_h)
                    if pt_f is None:
                        continue
                    u_f, v_f = pt_f 

                    pt_f_arr = np.array([(u_f, v_f)], dtype=np.float32).reshape(-1, 1, 2)
                    try:
                        rect_point = cv2.fisheye.undistortPoints(pt_f_arr, K_fish, D_fish, P=dis_K_rect)
                        mapped_pt = cv2.perspectiveTransform(rect_point,homography_BEV)
                        mx, my = int(mapped_pt[0][0][0]), int(mapped_pt[0][0][1])
                        p3d = map_2d_to_3d(mx, my, homography_BEV_to_3D)
                        p3d = [float(abs(float(x))) for x in p3d]
                    except Exception as e:
                        print(f"Transform error on view {i}: {e}")
                        continue

                    all_points_this_cycle.append(p3d)
                    all_fisheye_points.append((u_f, v_f))

                    tile_col = i % 2
                    tile_row = i // 2
                    x_in_grid = tile_col * rect_w + u_full
                    y_in_grid = tile_row * rect_h + v_full
                    all_rect_points_native.append((x_in_grid, y_in_grid))
                    print(f"VIEW {i+1} Class={cls} Conf={conf:.2f} | Rect=({u_full},{v_full}) | Fish=({u_f:.1f},{v_f:.1f}) | BEV=({mx},{my}) | 3D=({p3d[0]:.2f},{p3d[1]:.2f},{p3d[2]:.2f})")

            print(f"View {i+1}: {det_count} valid detections")

 
        for p3d in all_points_this_cycle:
            point_queue.put(p3d)

        rect_grid_native = make_rect_grid_disp_from_native(views)
        fisheye_display_resized = cv2.resize(frame_fish, (TARGET_WIDTH_FISHEYE, TARGET_HEIGHT))
        rect_grid_resized = cv2.resize(rect_grid_native, (TARGET_WIDTH_RECT, TARGET_HEIGHT))

        scale_fish_x = TARGET_WIDTH_FISHEYE / float(fisheye_w)
        scale_fish_y = TARGET_HEIGHT / float(fisheye_h)
        scale_rect_x = TARGET_WIDTH_RECT / float(rect_grid_native.shape[1])
        scale_rect_y = TARGET_HEIGHT / float(rect_grid_native.shape[0])

        for (u_f, v_f) in all_fisheye_points:
            u_fd = int(u_f * scale_fish_x)
            v_fd = int(v_f * scale_fish_y)
            cv2.circle(fisheye_display_resized, (u_fd, v_fd), 6, (0, 0, 255), -1)

        for (x_nat, y_nat) in all_rect_points_native:
            u_rd = int(x_nat * scale_rect_x)
            v_rd = int(y_nat * scale_rect_y)
            cv2.circle(rect_grid_resized, (u_rd, v_rd), 6, (0, 255, 0), -1)

        canvas = np.ones((TARGET_HEIGHT, TARGET_WIDTH_FISHEYE + TARGET_WIDTH_RECT, 3), dtype=np.uint8) 
        canvas[:, 0:TARGET_WIDTH_FISHEYE] = fisheye_display_resized
        canvas[:, TARGET_WIDTH_FISHEYE:] = rect_grid_resized

        cv2.imshow("Unified Mapping", canvas)
        key = cv2.waitKey(1)
        if key == 27:
            break

    cap_fish.release()
    for c in caps_rect:
        c.release()
    cv2.destroyAllWindows()

cv_thread = threading.Thread(target=run, daemon=True)
cv_thread.start()

try:
    while cv_thread.is_alive():
        while not point_queue.empty():
            p = point_queue.get()
            add_point_realtime(p)
        vis.poll_events()
        vis.update_renderer()
        time.sleep(0.03)
finally:
    vis.destroy_window()
    cv2.destroyAllWindows()


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Using device: cuda


C:\Users\Nahush\AppData\Local\Temp\ipykernel_14764\640523015.py:187: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  cls = int(box.cls.cpu().numpy()) if hasattr(box.cls, "cpu") else int(box.cls)
C:\Users\Nahush\AppData\Local\Temp\ipykernel_14764\640523015.py:195: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  conf = float(box.conf.cpu().numpy()) if hasattr(box.conf, "cpu") else float(box.conf)


VIEW 1 Class=1 Conf=0.98 | Rect=(845,504) | Fish=(1673.9,1309.0) | BEV=(1314,648) | 3D=(11.86,0.00,4.08)
View 1: 1 valid detections
View 2: 0 valid detections
View 3: 0 valid detections
VIEW 4 Class=1 Conf=0.98 | Rect=(323,585) | Fish=(1711.3,1204.0) | BEV=(1302,652) | 3D=(12.75,0.00,3.88)
VIEW 4 Class=0 Conf=0.95 | Rect=(964,214) | Fish=(1247.7,1742.0) | BEV=(1429,636) | 3D=(3.13,0.00,4.69)
View 4: 2 valid detections
VIEW 1 Class=1 Conf=0.98 | Rect=(845,505) | Fish=(1673.2,1308.6) | BEV=(1314,648) | 3D=(11.86,0.00,4.08)
View 1: 1 valid detections
View 2: 0 valid detections
View 3: 0 valid detections
VIEW 4 Class=1 Conf=0.98 | Rect=(323,585) | Fish=(1711.3,1204.0) | BEV=(1302,652) | 3D=(12.75,0.00,3.88)
VIEW 4 Class=0 Conf=0.95 | Rect=(958,214) | Fish=(1253.7,1741.7) | BEV=(1429,635) | 3D=(3.13,0.00,4.74)
View 4: 2 valid detections
VIEW 1 Class=1 Conf=0.98 | Rect=(845,505) | Fish=(1673.2,1308.6) | BEV=(1314,648) | 3D=(11.86,0.00,4.08)
View 1: 1 valid detections
View 2: 0 valid detectio

In [13]:
import numpy as np
from sklearn.cluster import DBSCAN

points = np.array([
    [10.0, 10.1],
    [10.2, 10.0],
    [10.3, 10.0],
    [10.1, 10.8],
    [10.3, 10.9]
])
labels = np.array([0, 1, 0, 1, 2])  

unique_classes = np.unique(labels)
cluster_results = {}

for cls in unique_classes:
    cls_points = points[labels == cls]
    db = DBSCAN(eps=0.9, min_samples=1).fit(cls_points)
    cluster_results[cls] = db.labels_

    print(f"Class {cls}: cluster labels -> {db.labels_}")


Class 0: cluster labels -> [0 0]
Class 1: cluster labels -> [0 0]
Class 2: cluster labels -> [0]


In [20]:
import numpy as np
import cv2
import open3d as o3d
import threading
from ultralytics import YOLO
from queue import Queue
import time
import torch

# ---------------- GPU SETUP ----------------
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f" Using device: {device}")

# ---------------- CAMERA SETTINGS ----------------
fisheye_params = {
    "width": 2592,
    "height": 1944,
    "cx": 1296.0,
    "cy": 972.0,
    "radius": 952.56,
    "theta_max_rad": 1.5708
}

cam_settings = [
    {"cx": 1296.0, "cy": 972.0, "fov": 90.0, "pitch": 58.0, "yaw": -18.0},
    {"cx": 1296.0, "cy": 972.0, "fov": 90.0, "pitch": 54.0, "yaw": 48.0},
    {"cx": 1296.0, "cy": 972.0, "fov": 90.0, "pitch": 28.0, "yaw": 150.0},
    {"cx": 1296.0, "cy": 972.0, "fov": 90.0, "pitch": 58.0, "yaw": 294.0},
]

views_path = [
    "recordings_2/camera_1.mp4",
    "recordings_2/camera_2.mp4",
    "recordings_2/camera_3.mp4",
    "recordings_2/camera_4.mp4"
]
fisheye_path = "recordings_2/fisheye.mp4"

# ---------------- YOLO MODEL ----------------
model = YOLO("16_09_25_SENd_v2.5_10s_960.pt")
model.to(device)
model.fuse()
model.half()

# ---------------- HOMOGRAPHIES ----------------
BEV = cv2.imread("ohlf.png")
homography_BEV = np.load("Homography for 229_camera_.npy")
homography_BEV_to_3D = np.load("Homography for bev to 3d.npy")

# ---------------- OPEN3D VISUALIZER ----------------
added_spheres = []
point_queue = Queue()
mesh = o3d.io.read_triangle_mesh("OHLF_obj/OHLF_v2.8.3 (1).obj", enable_post_processing=True)
mesh.compute_vertex_normals()
vis = o3d.visualization.Visualizer()
vis.create_window(window_name="3D Model", width=1280, height=720)
vis.add_geometry(mesh)

# ---------------- COLORS ----------------
CLASS_COLORS = {
    0: (0, 255, 0),
    1: (255, 0, 0),
    2: (0, 165, 255),
}
FONT = cv2.FONT_HERSHEY_SIMPLEX

# ---------------- GEOMETRY HELPERS ----------------
def compute_basis(yaw_deg, pitch_deg):
    yaw_deg_corrected = yaw_deg - 90.0
    yaw, pitch = np.deg2rad([yaw_deg_corrected, pitch_deg])
    Rz = np.array([[np.cos(-yaw), -np.sin(-yaw), 0],
                   [np.sin(-yaw), np.cos(-yaw), 0],
                   [0, 0, 1]])
    Rx = np.array([[1, 0, 0],
                   [0, np.cos(pitch), -np.sin(pitch)],
                   [0, np.sin(pitch), np.cos(pitch)]])
    return np.diag([1, -1, 1]) @ (Rx @ Rz)

def rect_to_fisheye_point(u, v, view_params, fisheye_params, rect_w, rect_h):
    Hr, Wr = rect_h, rect_w
    cx_r, cy_r = Wr / 2, Hr / 2
    fov = view_params["fov"]
    f = (Wr / 2) / np.tan(np.deg2rad(fov / 2))
    x = (u - cx_r) / f
    y = (v - cy_r) / f
    z = 1
    ray_rect = np.array([x, y, z])
    ray_rect /= np.linalg.norm(ray_rect)
    Rot = compute_basis(view_params["yaw"], view_params["pitch"])
    ray_fish = Rot.T @ ray_rect
    Xf, Yf, Zf = ray_fish
    theta = np.arccos(np.clip(Zf, -1, 1))
    phi = np.arctan2(Yf, Xf)
    radius, theta_max = fisheye_params["radius"], fisheye_params["theta_max_rad"]
    cx_f, cy_f = fisheye_params["cx"], fisheye_params["cy"]
    r = (theta / theta_max) * radius
    u_f = cx_f + r * np.cos(phi)
    v_f = cy_f - r * np.sin(phi)
    return (float(u_f), float(v_f))

def map_2d_to_3d(u, v, homography_bev_to_3d, y_value=0.0):
    src = np.array([u, v, 1.0], dtype=np.float64)
    dst = homography_bev_to_3d @ src
    dst /= dst[2]
    return np.array([dst[0], y_value, dst[1]])

def add_point_realtime(point3d):
    sphere = o3d.geometry.TriangleMesh.create_sphere(radius=0.25)
    sphere.paint_uniform_color([1, 0, 0])
    sphere.translate(point3d)
    vis.add_geometry(sphere, reset_bounding_box=False)
    vis.update_geometry(sphere)
    vis.poll_events()
    vis.update_renderer()
    added_spheres.append(sphere)

# ---------------- CAMERA MATRICES ----------------
dis_rect_w, dis_rect_h = 2592, 2592
fov = 160.0
fov_RAD = np.deg2rad(fov)
focal = (dis_rect_w / 2) / np.tan(fov_RAD / 2)
dis_K_rect = np.array([[focal, 0.0, dis_rect_w / 2],
                       [0.0, focal, dis_rect_h / 2],
                       [0.0, 0.0, 1.0]])

f_fish = fisheye_params["radius"] / fisheye_params["theta_max_rad"]
K_fish = np.array([[f_fish, 0.0, fisheye_params["cx"]],
                   [0.0, f_fish, fisheye_params["cy"]],
                   [0.0, 0.0, 1.0]])
D_fish = np.zeros((4, 1))

# ---------------- VIDEO STREAMS ----------------
TARGET_HEIGHT = 720
TARGET_WIDTH_FISHEYE = 720
TARGET_WIDTH_RECT = 720
cap_fish = cv2.VideoCapture(fisheye_path)
caps_rect = [cv2.VideoCapture(v) for v in views_path]

def read_frame_loop(cap):
    ret, frame = cap.read()
    if not ret:
        cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
        ret, frame = cap.read()
    return ret, frame

def make_rect_grid_disp_from_native(views_native):
    h, w = views_native[0].shape[:2]
    grid = np.ones((h * 2, w * 2, 3), dtype=np.uint8) * 0
    grid[0:h, 0:w] = views_native[0]
    grid[0:h, w:2*w] = views_native[1]
    grid[h:2*h, 0:w] = views_native[2]
    grid[h:2*h, w:2*w] = views_native[3]
    return grid

# ---------------- MAIN LOOP ----------------
def run():
    global vis
    frame_index = 0
    fps_display_time = time.time()

    while True:
        t0 = time.time()

        ret_f, frame_fish = read_frame_loop(cap_fish)
        if not ret_f:
            break

        # Read all 4 cameras
        views = []
        for c in caps_rect:
            ret, f = read_frame_loop(c)
            if not ret:
                break
            views.append(f)
        if len(views) != 4:
            break

        # Batch inference on all 4 views
        try:
            resized_views = [cv2.resize(v, (960, 540)) for v in views]
            results_list = model(resized_views, device=device, verbose=False)
        except Exception as e:
            print("Model error:", e)
            continue

        all_points_this_cycle = []
        all_fisheye_points = []
        all_rect_points_native = []

        for i, result in enumerate(results_list):
            frame = views[i]
            rect_h, rect_w = frame.shape[:2]
            det_count = 0

            if not hasattr(result, "boxes") or len(result.boxes) == 0:
                continue

            for box in result.boxes:
                cls = int(box.cls.cpu().numpy()) if hasattr(box.cls, "cpu") else int(box.cls)
                if cls not in [0, 1, 2]:
                    continue

                conf = float(box.conf.cpu().numpy()) if hasattr(box.conf, "cpu") else float(box.conf)
                x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                scale_x = rect_w / 960.0
                scale_y = rect_h / 540.0
                x1, x2 = x1 * scale_x, x2 * scale_x
                y1, y2 = y1 * scale_y, y2 * scale_y

                color = CLASS_COLORS.get(cls, (0, 255, 255))
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)
                det_count += 1

                u_full = int((x1 + x2) / 2.0)
                v_full = int(y2)

                pt_f = rect_to_fisheye_point(u_full, v_full, cam_settings[i], fisheye_params, rect_w, rect_h)
                if pt_f is None:
                    continue
                u_f, v_f = pt_f

                pt_f_arr = np.array([(u_f, v_f)], dtype=np.float32).reshape(-1, 1, 2)
                try:
                    rect_point = cv2.fisheye.undistortPoints(pt_f_arr, K_fish, D_fish, P=dis_K_rect)
                    mapped_pt = cv2.perspectiveTransform(rect_point, homography_BEV)
                    mx, my = int(mapped_pt[0][0][0]), int(mapped_pt[0][0][1])
                    p3d = map_2d_to_3d(mx, my, homography_BEV_to_3D)
                    p3d = [float(abs(float(x))) for x in p3d]
                except Exception as e:
                    print(f"Transform error on view {i}: {e}")
                    continue

                all_points_this_cycle.append(p3d)
                all_fisheye_points.append((u_f, v_f))

                tile_col = i % 2
                tile_row = i // 2
                x_in_grid = tile_col * rect_w + u_full
                y_in_grid = tile_row * rect_h + v_full
                all_rect_points_native.append((x_in_grid, y_in_grid))

            print(f"View {i+1}: {det_count} detections")

        for p3d in all_points_this_cycle:
            point_queue.put(p3d)

        # --- Display ---
        rect_grid_native = make_rect_grid_disp_from_native(views)
        fisheye_display_resized = cv2.resize(frame_fish, (TARGET_WIDTH_FISHEYE, TARGET_HEIGHT))
        rect_grid_resized = cv2.resize(rect_grid_native, (TARGET_WIDTH_RECT, TARGET_HEIGHT))
        canvas = np.ones((TARGET_HEIGHT, TARGET_WIDTH_FISHEYE + TARGET_WIDTH_RECT, 3), dtype=np.uint8)
        canvas[:, 0:TARGET_WIDTH_FISHEYE] = fisheye_display_resized
        canvas[:, TARGET_WIDTH_FISHEYE:] = rect_grid_resized

        # FPS display
        fps = 1.0 / (time.time() - t0)
        cv2.putText(canvas, f"FPS: {fps:.2f}", (30, 40), FONT, 1, (0, 255, 0), 2)

        # Limit OpenCV display rate
        if frame_index % 3 == 0:
            cv2.imshow("Unified Mapping", canvas)

        frame_index += 1
        if cv2.waitKey(1) == 27:
            break

    cap_fish.release()
    for c in caps_rect:
        c.release()
    cv2.destroyAllWindows()

# ---------------- THREAD MANAGEMENT ----------------
cv_thread = threading.Thread(target=run, daemon=True)
cv_thread.start()

try:
    while cv_thread.is_alive():
        while not point_queue.empty():
            p = point_queue.get()
            add_point_realtime(p)
        vis.poll_events()
        vis.update_renderer()
        time.sleep(0.02)
finally:
    vis.destroy_window()
    cv2.destroyAllWindows()


 Using device: cuda
YOLOv10s summary (fused): 106 layers, 7,219,161 parameters, 0 gradients, 21.4 GFLOPs


C:\Users\Nahush\AppData\Local\Temp\ipykernel_14764\2080884422.py:199: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  cls = int(box.cls.cpu().numpy()) if hasattr(box.cls, "cpu") else int(box.cls)
C:\Users\Nahush\AppData\Local\Temp\ipykernel_14764\2080884422.py:203: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  conf = float(box.conf.cpu().numpy()) if hasattr(box.conf, "cpu") else float(box.conf)


View 1: 1 detections
View 4: 2 detections
View 1: 1 detections
View 4: 2 detections
View 1: 1 detections
View 4: 2 detections
View 1: 1 detections
View 4: 2 detections
View 1: 1 detections
View 4: 2 detections
View 1: 1 detections
View 4: 2 detections
View 1: 1 detections
View 4: 2 detections
View 1: 1 detections
View 4: 2 detections
View 1: 1 detections
View 4: 2 detections
View 1: 1 detections
View 4: 2 detections
View 1: 1 detections
View 4: 2 detections
View 1: 1 detections
View 4: 2 detections
View 1: 1 detections
View 4: 2 detections
View 1: 1 detections
View 4: 2 detections
View 1: 1 detections
View 4: 2 detections
View 1: 1 detections
View 4: 2 detections
View 1: 1 detections
View 4: 2 detections
View 1: 1 detections
View 4: 2 detections
View 1: 1 detections
View 4: 2 detections
View 1: 1 detections
View 4: 2 detections
View 1: 1 detections
View 4: 2 detections
View 1: 1 detections
View 4: 2 detections
View 1: 1 detections
View 4: 2 detections
View 1: 1 detections
View 4: 2 det

In [18]:
import numpy as np
import cv2
import open3d as o3d
import threading
from ultralytics import YOLO
from queue import Queue
import time
import torch

# ---------------- GPU SETUP ----------------
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f" Using device: {device}")

# ---------------- CAMERA SETTINGS ----------------
fisheye_params = {
    "width": 2592,
    "height": 1944,
    "cx": 1296.0,
    "cy": 972.0,
    "radius": 952.56,
    "theta_max_rad": 1.5708
}

cam_settings = [
    {"cx": 1296.0, "cy": 972.0, "fov": 90.0, "pitch": 58.0, "yaw": -18.0},
    {"cx": 1296.0, "cy": 972.0, "fov": 90.0, "pitch": 54.0, "yaw": 48.0},
    {"cx": 1296.0, "cy": 972.0, "fov": 90.0, "pitch": 28.0, "yaw": 150.0},
    {"cx": 1296.0, "cy": 972.0, "fov": 90.0, "pitch": 58.0, "yaw": 294.0},
]

views_path = [
    "recordings_3/camera_1.mp4",
    "recordings_3/camera_2.mp4",
    "recordings_3/camera_3.mp4",
    "recordings_3/camera_4.mp4"
]
fisheye_path = "recordings_3/fisheye.mp4"

# ---------------- YOLO MODEL ----------------
model = YOLO("16_09_25_SENd_v2.5_10s_960.pt")
model.to(device)
model.fuse()
model.half()
print(" YOLO model loaded and fused for inference.")

# ---------------- HOMOGRAPHIES ----------------
BEV = cv2.imread("ohlf.png")
homography_BEV = np.load("Homography for 229_camera_.npy")
homography_BEV_to_3D = np.load("Homography for bev to 3d.npy")

# ---------------- OPEN3D VISUALIZER ----------------
point_queue = Queue()
mesh = o3d.io.read_triangle_mesh("OHLF_obj/OHLF_v2.8.3 (1).obj", enable_post_processing=True)
mesh.compute_vertex_normals()
vis = o3d.visualization.Visualizer()
vis.create_window(window_name="3D Model", width=1280, height=720)
vis.add_geometry(mesh)
print(" Open3D visualizer initialized.")
current_spheres = []

# ---------------- COLORS ----------------
CLASS_COLORS = {
    0: (0, 255, 0),
    1: (255, 0, 0),
    2: (0, 165, 255),
}
FONT = cv2.FONT_HERSHEY_SIMPLEX

# ---------------- GEOMETRY HELPERS ----------------
def compute_basis(yaw_deg, pitch_deg):
    yaw_deg_corrected = yaw_deg - 90.0
    yaw, pitch = np.deg2rad([yaw_deg_corrected, pitch_deg])
    Rz = np.array([[np.cos(-yaw), -np.sin(-yaw), 0],
                   [np.sin(-yaw), np.cos(-yaw), 0],
                   [0, 0, 1]])
    Rx = np.array([[1, 0, 0],
                   [0, np.cos(pitch), -np.sin(pitch)],
                   [0, np.sin(pitch), np.cos(pitch)]])
    return np.diag([1, -1, 1]) @ (Rx @ Rz)

def rect_to_fisheye_point(u, v, view_params, fisheye_params, rect_w, rect_h):
    Hr, Wr = rect_h, rect_w
    cx_r, cy_r = Wr / 2, Hr / 2
    fov = view_params["fov"]
    f = (Wr / 2) / np.tan(np.deg2rad(fov / 2))
    x = (u - cx_r) / f
    y = (v - cy_r) / f
    z = 1
    ray_rect = np.array([x, y, z])
    ray_rect /= np.linalg.norm(ray_rect)
    Rot = compute_basis(view_params["yaw"], view_params["pitch"])
    ray_fish = Rot.T @ ray_rect
    Xf, Yf, Zf = ray_fish
    theta = np.arccos(np.clip(Zf, -1, 1))
    phi = np.arctan2(Yf, Xf)
    radius, theta_max = fisheye_params["radius"], fisheye_params["theta_max_rad"]
    cx_f, cy_f = fisheye_params["cx"], fisheye_params["cy"]
    r = (theta / theta_max) * radius
    u_f = cx_f + r * np.cos(phi)
    v_f = cy_f - r * np.sin(phi)
    return (float(u_f), float(v_f))

def map_2d_to_3d(u, v, homography_bev_to_3d, y_value=0.0):
    src = np.array([u, v, 1.0], dtype=np.float64)
    dst = homography_bev_to_3d @ src
    dst /= dst[2]
    return np.array([dst[0], y_value, dst[1]])

# --- Update spheres dynamically ---
def update_spheres(points_3d):
    global vis, current_spheres
    for s in current_spheres:
        vis.remove_geometry(s, reset_bounding_box=False)
    current_spheres = []

    for p in points_3d:
        sphere = o3d.geometry.TriangleMesh.create_sphere(radius=0.25)
        sphere.paint_uniform_color([1, 0, 0])
        sphere.translate(p)
        vis.add_geometry(sphere, reset_bounding_box=False)
        current_spheres.append(sphere)

    vis.poll_events()
    vis.update_renderer()

# ---------------- CAMERA MATRICES ----------------
dis_rect_w, dis_rect_h = 2592, 2592
fov = 160.0
fov_RAD = np.deg2rad(fov)
focal = (dis_rect_w / 2) / np.tan(fov_RAD / 2)
dis_K_rect = np.array([[focal, 0.0, dis_rect_w / 2],
                       [0.0, focal, dis_rect_h / 2],
                       [0.0, 0.0, 1.0]])

f_fish = fisheye_params["radius"] / fisheye_params["theta_max_rad"]
K_fish = np.array([[f_fish, 0.0, fisheye_params["cx"]],
                   [0.0, f_fish, fisheye_params["cy"]],
                   [0.0, 0.0, 1.0]])
D_fish = np.zeros((4, 1))

# ---------------- VIDEO STREAMS ----------------
TARGET_HEIGHT = 720
TARGET_WIDTH_FISHEYE = 720
TARGET_WIDTH_RECT = 720
cap_fish = cv2.VideoCapture(fisheye_path)
caps_rect = [cv2.VideoCapture(v) for v in views_path]

def read_frame_loop(cap):
    ret, frame = cap.read()
    if not ret:
        cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
        time.sleep(0.05)
        ret, frame = cap.read()
    return ret, frame

def make_rect_grid_disp_from_native(views_native):
    h, w = views_native[0].shape[:2]
    grid = np.ones((h * 2, w * 2, 3), dtype=np.uint8) * 0
    grid[0:h, 0:w] = views_native[0]
    grid[0:h, w:2*w] = views_native[1]
    grid[h:2*h, 0:w] = views_native[2]
    grid[h:2*h, w:2*w] = views_native[3]
    return grid

# ---------------- MAIN LOOP ----------------
def run():
    global vis
    frame_index = 0

    while True:
        t0 = time.time()

        ret_f, frame_fish = read_frame_loop(cap_fish)
        if not ret_f:
            break

        # Read all 4 cameras
        views = []
        for c in caps_rect:
            ret, f = read_frame_loop(c)
            if not ret:
                break
            views.append(f)
        if len(views) != 4:
            continue

        # Resize safely
        valid_views = []
        for v in views:
            if v is None or v.size == 0:
                valid_views = []
                break
            valid_views.append(cv2.resize(v, (960, 540)))

        if len(valid_views) != 4:
            continue

        # YOLO Inference
        try:
            with torch.inference_mode():
                results_list = model(valid_views, device=device, verbose=False)
        except Exception as e:
            print(f" Model error during batch inference: {e}")
            continue

        all_points_this_cycle = []
        all_fisheye_points = []
        cls_0 = []
        cls_1 = []
        cls_2 = []
        for i, result in enumerate(results_list):
            frame = views[i]
            rect_h, rect_w = frame.shape[:2]

            if not hasattr(result, "boxes") or len(result.boxes) == 0:
                continue

            for box in result.boxes:
                cls = int(box.cls.item())
                if cls not in [0, 1, 2]:
                    continue

                x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                scale_x, scale_y = rect_w / 960.0, rect_h / 540.0
                x1, x2, y1, y2 = x1 * scale_x, x2 * scale_x, y1 * scale_y, y2 * scale_y
                color = CLASS_COLORS.get(cls, (0, 255, 255))
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)

                u_full, v_full = int((x1 + x2) / 2.0), int(y2)
                pt_f = rect_to_fisheye_point(u_full, v_full, cam_settings[i], fisheye_params, rect_w, rect_h)
                if pt_f is None:
                    continue

                u_f, v_f = pt_f
                all_fisheye_points.append((u_f, v_f))

                pt_f_arr = np.array([(u_f, v_f)], dtype=np.float32).reshape(-1, 1, 2)
                try:
                    rect_point = cv2.fisheye.undistortPoints(pt_f_arr, K_fish, D_fish, P=dis_K_rect)
                    mapped_pt = cv2.perspectiveTransform(rect_point, homography_BEV)
                    mx, my = int(mapped_pt[0][0][0]), int(mapped_pt[0][0][1])
                    p3d = map_2d_to_3d(mx, my, homography_BEV_to_3D)
                    all_points_this_cycle.append(np.abs(p3d))
                    print(f"cls: ({cls}), Rect:({u_full},{v_full}), fisheye:({u_f},{v_f}), 3d: ({p3d})")
                    if cls == 0:
                        cls_0.append(p3d)
                    elif cls == 1:
                        cls_1.append(p3d)
                    elif cls == 2:
                        cls_2.append(p3d)

                except Exception as e:
                    print(f"Transform error view {i}: {e}")
                    continue
        print(cls_0)
        print(cls_1)
        # Draw detections on fisheye
        for (u_f, v_f) in all_fisheye_points:
            cv2.circle(frame_fish, (int(u_f), int(v_f)), 5, (0, 0, 255), -1)

        if all_points_this_cycle:
            point_queue.put(("update_3d", np.array(all_points_this_cycle)))

        # Display
        rect_grid_native = make_rect_grid_disp_from_native(views)
        fisheye_resized = cv2.resize(frame_fish, (TARGET_WIDTH_FISHEYE, TARGET_HEIGHT))
        rect_resized = cv2.resize(rect_grid_native, (TARGET_WIDTH_RECT, TARGET_HEIGHT))
        canvas = np.ones((TARGET_HEIGHT, TARGET_WIDTH_FISHEYE + TARGET_WIDTH_RECT, 3), dtype=np.uint8)
        canvas[:, 0:TARGET_WIDTH_FISHEYE] = fisheye_resized
        canvas[:, TARGET_WIDTH_FISHEYE:] = rect_resized

        fps = 1.0 / (time.time() - t0)
        cv2.putText(canvas, f"FPS: {fps:.2f}", (30, 40), FONT, 1, (0, 255, 0), 2)

        if frame_index % 3 == 0:
            cv2.imshow("Unified Mapping", canvas)

        frame_index += 1
        if cv2.waitKey(1) == 27:
            break

    cap_fish.release()
    for c in caps_rect:
        c.release()
    cv2.destroyAllWindows()

# ---------------- THREAD MANAGEMENT ----------------
cv_thread = threading.Thread(target=run, daemon=True)
cv_thread.start()

try:
    while cv_thread.is_alive():
        while not point_queue.empty():
            msg_type, payload = point_queue.get()
            if msg_type == "update_3d":
                update_spheres(payload)
        vis.poll_events()
        vis.update_renderer()
        time.sleep(0.02)
finally:
    vis.destroy_window()
    cv2.destroyAllWindows()


 Using device: cuda
YOLOv10s summary (fused): 106 layers, 7,219,161 parameters, 0 gradients, 21.4 GFLOPs
 YOLO model loaded and fused for inference.
 Open3D visualizer initialized.
cls: (1), Rect:(845,503), fisheye:(1674.7078099679752,1309.4327928111782), 3d: ([11.86087805  0.          4.07872218])
cls: (0), Rect:(852,578), fisheye:(1615.9675409850495,1285.243546741993), 3d: ([11.9343245   0.          3.41126196])
cls: (1), Rect:(321,588), fisheye:(1711.1512599924051,1201.2595651617303), 3d: ([12.75447137  0.          3.82448028])
cls: (0), Rect:(417,618), fisheye:(1630.5969425959215,1226.190221294028), 3d: ([12.38218965  0.          3.2585243 ])
[array([11.9343245 ,  0.        ,  3.41126196]), array([12.38218965,  0.        ,  3.2585243 ])]
[array([11.86087805,  0.        ,  4.07872218]), array([12.75447137,  0.        ,  3.82448028])]
cls: (1), Rect:(845,503), fisheye:(1674.7078099679752,1309.4327928111782), 3d: ([11.86087805  0.          4.07872218])
cls: (0), Rect:(854,578), fishey

In [21]:
import numpy as np
import cv2
import open3d as o3d
import threading
from ultralytics import YOLO
from queue import Queue
import time
import torch

# ---------------- GPU SETUP ----------------
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f" Using device: {device}")

# ---------------- CAMERA SETTINGS ----------------
fisheye_params = {
    "width": 2592,
    "height": 1944,
    "cx": 1296.0,
    "cy": 972.0,
    "radius": 952.56,
    "theta_max_rad": 1.5708
}

cam_settings = [
    {"cx": 1296.0, "cy": 972.0, "fov": 90.0, "pitch": 58.0, "yaw": -18.0},
    {"cx": 1296.0, "cy": 972.0, "fov": 90.0, "pitch": 54.0, "yaw": 48.0},
    {"cx": 1296.0, "cy": 972.0, "fov": 90.0, "pitch": 28.0, "yaw": 150.0},
    {"cx": 1296.0, "cy": 972.0, "fov": 90.0, "pitch": 58.0, "yaw": 294.0},
]

views_path = [
    "recordings_3/camera_1.mp4",
    "recordings_3/camera_2.mp4",
    "recordings_3/camera_3.mp4",
    "recordings_3/camera_4.mp4"
]
fisheye_path = "recordings_3/fisheye.mp4"

# ---------------- YOLO MODEL ----------------
model = YOLO("16_09_25_SENd_v2.5_10s_960.pt")
model.to(device)
model.fuse()
model.half()
print(" YOLO model loaded and fused for inference.")

# ---------------- HOMOGRAPHIES ----------------
BEV = cv2.imread("ohlf.png")
homography_BEV = np.load("Homography for 229_camera_.npy")
homography_BEV_to_3D = np.load("Homography for bev to 3d.npy")

# ---------------- OPEN3D VISUALIZER ----------------
point_queue = Queue()
mesh = o3d.io.read_triangle_mesh("OHLF_obj/OHLF_v2.8.3 (1).obj", enable_post_processing=True)
mesh.compute_vertex_normals()
vis = o3d.visualization.Visualizer()
vis.create_window(window_name="3D Model", width=1280, height=720)
vis.add_geometry(mesh)
print(" Open3D visualizer initialized.")
current_spheres = []

# ---------------- COLORS ----------------
CLASS_COLORS = {
    0: (0, 255, 0),
    1: (255, 0, 0),
    2: (0, 165, 255),
}
FONT = cv2.FONT_HERSHEY_SIMPLEX

# ---------------- GEOMETRY HELPERS ----------------
def compute_basis(yaw_deg, pitch_deg):
    yaw_deg_corrected = yaw_deg - 90.0
    yaw, pitch = np.deg2rad([yaw_deg_corrected, pitch_deg])
    Rz = np.array([[np.cos(-yaw), -np.sin(-yaw), 0],
                   [np.sin(-yaw), np.cos(-yaw), 0],
                   [0, 0, 1]])
    Rx = np.array([[1, 0, 0],
                   [0, np.cos(pitch), -np.sin(pitch)],
                   [0, np.sin(pitch), np.cos(pitch)]])
    return np.diag([1, -1, 1]) @ (Rx @ Rz)

def rect_to_fisheye_point(u, v, view_params, fisheye_params, rect_w, rect_h):
    Hr, Wr = rect_h, rect_w
    cx_r, cy_r = Wr / 2, Hr / 2
    fov = view_params["fov"]
    f = (Wr / 2) / np.tan(np.deg2rad(fov / 2))
    x = (u - cx_r) / f
    y = (v - cy_r) / f
    z = 1
    ray_rect = np.array([x, y, z])
    ray_rect /= np.linalg.norm(ray_rect)
    Rot = compute_basis(view_params["yaw"], view_params["pitch"])
    ray_fish = Rot.T @ ray_rect
    Xf, Yf, Zf = ray_fish
    theta = np.arccos(np.clip(Zf, -1, 1))
    phi = np.arctan2(Yf, Xf)
    radius, theta_max = fisheye_params["radius"], fisheye_params["theta_max_rad"]
    cx_f, cy_f = fisheye_params["cx"], fisheye_params["cy"]
    r = (theta / theta_max) * radius
    u_f = cx_f + r * np.cos(phi)
    v_f = cy_f - r * np.sin(phi)
    return (float(u_f), float(v_f))

def map_2d_to_3d(u, v, homography_bev_to_3d, y_value=0.0):
    src = np.array([u, v, 1.0], dtype=np.float64)
    dst = homography_bev_to_3d @ src
    dst /= dst[2]
    return np.array([dst[0], y_value, dst[1]])

# --- Update spheres dynamically ---
def update_spheres(points_3d):
    global vis, current_spheres
    for s in current_spheres:
        vis.remove_geometry(s, reset_bounding_box=False)
    current_spheres = []

    for (c,p) in points_3d:
        sphere = o3d.geometry.TriangleMesh.create_sphere(radius=0.25)
        if c == 0:
            sphere.paint_uniform_color([1, 0, 0])
        elif c == 1:
            sphere.paint_uniform_color([0, 1, 0])
        sphere.translate(p)
        vis.add_geometry(sphere, reset_bounding_box=False)
        current_spheres.append(sphere)

    vis.poll_events()
    vis.update_renderer()

# ---------------- CAMERA MATRICES ----------------
dis_rect_w, dis_rect_h = 2592, 2592
fov = 160.0
fov_RAD = np.deg2rad(fov)
focal = (dis_rect_w / 2) / np.tan(fov_RAD / 2)
dis_K_rect = np.array([[focal, 0.0, dis_rect_w / 2],
                       [0.0, focal, dis_rect_h / 2],
                       [0.0, 0.0, 1.0]])

f_fish = fisheye_params["radius"] / fisheye_params["theta_max_rad"]
K_fish = np.array([[f_fish, 0.0, fisheye_params["cx"]],
                   [0.0, f_fish, fisheye_params["cy"]],
                   [0.0, 0.0, 1.0]])
D_fish = np.zeros((4, 1))

# ---------------- VIDEO STREAMS ----------------
TARGET_HEIGHT = 720
TARGET_WIDTH_FISHEYE = 720
TARGET_WIDTH_RECT = 720
cap_fish = cv2.VideoCapture(fisheye_path)
caps_rect = [cv2.VideoCapture(v) for v in views_path]

def read_frame_loop(cap):
    ret, frame = cap.read()
    if not ret:
        cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
        time.sleep(0.05)
        ret, frame = cap.read()
    return ret, frame

def make_rect_grid_disp_from_native(views_native):
    h, w = views_native[0].shape[:2]
    grid = np.ones((h * 2, w * 2, 3), dtype=np.uint8) * 0
    grid[0:h, 0:w] = views_native[0]
    grid[0:h, w:2*w] = views_native[1]
    grid[h:2*h, 0:w] = views_native[2]
    grid[h:2*h, w:2*w] = views_native[3]
    return grid

# ---------------- MAIN LOOP ----------------
def run():
    global vis
    frame_index = 0

    while True:
        t0 = time.time()

        ret_f, frame_fish = read_frame_loop(cap_fish)
        if not ret_f:
            break

        # Read all 4 cameras
        views = []
        for c in caps_rect:
            ret, f = read_frame_loop(c)
            if not ret:
                break
            views.append(f)
        if len(views) != 4:
            continue

        # Resize safely
        valid_views = []
        for v in views:
            if v is None or v.size == 0:
                valid_views = []
                break
            valid_views.append(cv2.resize(v, (960, 540)))

        if len(valid_views) != 4:
            continue

        # YOLO Inference
        try:
            with torch.inference_mode():
                results_list = model(valid_views, device=device, verbose=False)
        except Exception as e:
            print(f" Model error during batch inference: {e}")
            continue

        all_points_this_cycle = []
        all_fisheye_points = []
        
        for i, result in enumerate(results_list):
            frame = views[i]
            rect_h, rect_w = frame.shape[:2]

            if not hasattr(result, "boxes") or len(result.boxes) == 0:
                continue

            for box in result.boxes:
                cls = int(box.cls.item())
                if cls not in [0, 1, 2]:
                    continue

                x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                scale_x, scale_y = rect_w / 960.0, rect_h / 540.0
                x1, x2, y1, y2 = x1 * scale_x, x2 * scale_x, y1 * scale_y, y2 * scale_y
                color = CLASS_COLORS.get(cls, (0, 255, 255))
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)

                u_full, v_full = int((x1 + x2) / 2.0), int(y2)
                pt_f = rect_to_fisheye_point(u_full, v_full, cam_settings[i], fisheye_params, rect_w, rect_h)
                if pt_f is None:
                    continue
                all_fisheye_points.append([cls,pt_f[0],pt_f[1]])
        
        class_ = all_fisheye_points[:,0].astype(int)
        uv = all_fisheye_points[:,1:]
        unique_class = np.unique(class_)
        mean_fisheye_points = []
        for c in unique_class:
            cls_points = uv[cls==c]
            if c == 0:
                eps = 65
                min_samples = 1
            elif c == 1:
                eps = 120
                min_samples = 1


            db = DBSCAN(eps=eps, min_samples=min_samples).fit(cls_points)
            labels = db.labels_
            unique_label = np.unique(labels)
            for cluster_id in unique_label:
                if cluster_id == -1:
                    continue
                cluster_points = cls_points[labels == cluster_id]
                centroid = cluster_points.mean(axis=0)
                mean_fisheye_points.append((c,centroid[0], centroid[1]))
                pt_f_arr = np.array([(centroid[0], centroid[1])], dtype=np.float32).reshape(-1, 1, 2)
                try:
                    rect_point = cv2.fisheye.undistortPoints(pt_f_arr, K_fish, D_fish, P=dis_K_rect)
                    mapped_pt = cv2.perspectiveTransform(rect_point, homography_BEV)
                    mx, my = int(mapped_pt[0][0][0]), int(mapped_pt[0][0][1])
                    p3d = map_2d_to_3d(mx, my, homography_BEV_to_3D)
                    all_points_this_cycle.append((c,np.abs(p3d)))
                    print(f"cls: ({c}), fisheye:({centroid[0]},{centroid[1]}), 3d: ({p3d})")

                except Exception as e:
                    print(f"Transform error view {i}: {e}")
                    continue

        # Draw detections on fisheye
        for (c,u_f, v_f) in mean_fisheye_points:
            if c == 0:
                cv2.circle(frame_fish, (int(u_f), int(v_f)), 5, (0, 0, 255), -1)
            elif c == 1:
                cv2.circle(frame_fish, (int(u_f), int(v_f)), 5, (0, 255, 0), -1)

        if all_points_this_cycle:
            point_queue.put(("update_3d", np.array(all_points_this_cycle)))

        # Display
        rect_grid_native = make_rect_grid_disp_from_native(views)
        fisheye_resized = cv2.resize(frame_fish, (TARGET_WIDTH_FISHEYE, TARGET_HEIGHT))
        rect_resized = cv2.resize(rect_grid_native, (TARGET_WIDTH_RECT, TARGET_HEIGHT))
        canvas = np.ones((TARGET_HEIGHT, TARGET_WIDTH_FISHEYE + TARGET_WIDTH_RECT, 3), dtype=np.uint8)
        canvas[:, 0:TARGET_WIDTH_FISHEYE] = fisheye_resized
        canvas[:, TARGET_WIDTH_FISHEYE:] = rect_resized

        fps = 1.0 / (time.time() - t0)
        cv2.putText(canvas, f"FPS: {fps:.2f}", (30, 40), FONT, 1, (0, 255, 0), 2)

        if frame_index % 3 == 0:
            cv2.imshow("Unified Mapping", canvas)

        frame_index += 1
        if cv2.waitKey(1) == 27:
            break

    cap_fish.release()
    for c in caps_rect:
        c.release()
    cv2.destroyAllWindows()

# ---------------- THREAD MANAGEMENT ----------------
cv_thread = threading.Thread(target=run, daemon=True)
cv_thread.start()

try:
    while cv_thread.is_alive():
        while not point_queue.empty():
            msg_type, payload = point_queue.get()
            if msg_type == "update_3d":
                update_spheres(payload)
        vis.poll_events()
        vis.update_renderer()
        time.sleep(0.02)
finally:
    vis.destroy_window()
    cv2.destroyAllWindows()


 Using device: cuda
YOLOv10s summary (fused): 106 layers, 7,219,161 parameters, 0 gradients, 21.4 GFLOPs
 YOLO model loaded and fused for inference.
 Open3D visualizer initialized.


Exception in thread Thread-248 (run):
Traceback (most recent call last):
  File "c:\Users\Nahush\anaconda3\envs\fisheye_env\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "c:\Users\Nahush\anaconda3\envs\fisheye_env\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Nahush\AppData\Local\Temp\ipykernel_3620\2684496033.py", line 236, in run
TypeError: list indices must be integers or slices, not tuple
